# Cryptocurrency Analytics

## Import Libraries

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import networkx as nx
from pyvis import network as net
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup
import time
from IPython.display import display
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings("ignore")


## Graph Network

### Prepare Data

In [2]:
# Load data from balances
network_data=pd.read_csv('C:/Users/JCRG/Desktop/Master/Dissertation/Data/Transactional_Network_Ether.csv')
network_data.head()

,from_address,to_address,block_timestamp,value_ether
0,0x7ac34681f6aaeb691e150c43ee494177c0e2c183,0x7fb6fefe6c524d9cb06025583c0ad59aad2f6f5b,2018-07-11 11:57:20 UTC,1.00000
1,0xf1cf296c11d3b010e6dd3d48b6c5aaed613e96f1,0x2ec2c8729da460eb9743e86c3d16713f6aa47325,2018-07-11 01:53:49 UTC,3.04768
2,0x8e04af7f7c76daa9ab429b1340e0327b5b835748,0x0000000000085d4780b73119b644ae5ecd22b376,2019-08-20 14:27:26 UTC,0.00000
3,0x8f6147a13514d7cc7c1c85bc4fad8f7f1bc63b57,0x0d8775f648430679a709e98d2b0cb6250d2887ef,2019-05-30 03:04:18 UTC,0.00000
4,0x7b6f730249555a85d39327b90f0080e9dc63daff,0x1cb9ede1a43ab0265d6c44be4f3db554abf822c5,2019-05-30 13:17:56 UTC,250.00000


In [ ]:
# Prepare data for network. Group by sender and receiver and count the number of transactions
network_data_grouped=network_data[['from_address','to_address','value_ether']].groupby(['from_address','to_address']).count()
network_data_grouped.reset_index(inplace=True)
network_data_grouped.sort_values(by=['value_ether'],ascending=False,inplace=True)
network_data_grouped.head(10)

### Generate Dataframe Output

In [ ]:
# Create object with network data
G = nx.from_pandas_edgelist(network_data_grouped,
                            source='from_address',
                            target='to_address',
                            create_using=nx.DiGraph)

In [ ]:
# Get the nodes where the top 10,000 whales are transacting with.
df=pd.DataFrame(G.in_degree()).sort_values(1,ascending=False)
df.columns=['Address','Count']
# Select the top 100 addresses
df=df[:99]

In [ ]:
# Get nametag with the addresses from Etherscan through crawler. This is perform to identify easier which are the names of the addresses, if available.
def get_nametag(address):
    #Make API request to Etherscan
    url = 'https://etherscan.io/address/%s'%(address) # Initiate with the first page of accounts with 100 records
    req = Request(url,headers={'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'})   # Etherscan restricts the access for Webscrapping, reason why headers needs to be stated
    response = urlopen(req, timeout=500).read() # Read response
    page_soup = soup(response, "html.parser") # Parse HTML response
    page_soup.find('title')
    a=page_soup.find('title')
    a=str(a)
    a=((a.split('|')[0])[10:])[:-1]
    time.sleep(2) # Etherscan restricts the access if the requests are perform too fast. The function delays the code by two seconds
    return a

In [ ]:
# Create column with nametag
df['tag'] = df['Address'].apply(get_nametag)
# Visualize dataframe
with pd.option_context('display.max_rows', 100, 'display.max_columns', 10):
    display(df)

## Insider Trading Preparation

### Import and preprocess Listing Dates

In [ ]:
# Import Listings from Binance
listings=pd.read_csv('Listing_Dates.csv')
listings=df.merge(listings,on='tag',how='left')
listings.columns=['to_address','count','tag','listing_date']

In [ ]:
# Merge data with the transactional data from addresses and filter out the addresses that are not in the listings
df2=network_data.merge(listings,on='to_address')
df2=df2[(df2['listing_date']!='Not listed')&(~df2['listing_date'].isna())]
# Configure time format for listing date
df2['listing_date']=pd.to_datetime(df2['listing_date'],infer_datetime_format=True)
# Configure time format for block_timestamp date
df2['block_timestamp']=pd.to_datetime(df2['block_timestamp'],infer_datetime_format=True)
df2['block_timestamp']=df2['block_timestamp'].dt.date
df2['block_timestamp']=pd.to_datetime(df2['block_timestamp'],infer_datetime_format=True)

### Create Filters in Transactional data with Listings In Binance

In [ ]:
# Create column with True & False for the difference between the listing date and the block timestamp. This are the transactions that has been done before 30 days of the listing in Binance.
df2['date_before_listing']=(df2['block_timestamp'] < (df2['listing_date']-timedelta(days=30)))
# Create column with True & False for transactions that has been done between the two stated dates, which are between the listing date and 30 days before it.
df2['date_between_listing']=(df2['block_timestamp'] > (df2['listing_date']-timedelta(days=30))) & (df2['block_timestamp'] < df2['listing_date'])
# Create column with key of address and tag for future filtering
df2['key']=df2[['from_address','tag']].agg('-'.join,axis=1)

In [ ]:
# Create dataframe with 'from_address' and 'to_address' to filter 'from_address' that have had transactions 30 days before the listing in Binance.
previous_transactions_true=df2[df2['date_before_listing']==True].groupby(['from_address','tag']).nunique()
previous_transactions_true=previous_transactions_true.reset_index()
previous_transactions_true=previous_transactions_true[['from_address','tag']]
# Create column with key of address and tag for future filtering
previous_transactions_true['key']=previous_transactions_true[['from_address','tag']].agg('-'.join,axis=1)

In [ ]:
# Create dataframe with filter of transactional data and listings
df3 = df2[~df2.key.isin(list(previous_transactions_true['key']))]
df3=df3[df3['date_between_listing']==True]
df3.groupby(['from_address','tag']).nunique()
# Print
df3

In [ ]:
df3.groupby(['from_address','tag']).count()

### Export Data

In [ ]:
df3.to_csv("df3.csv",index=False)
